In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [2]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
users_eliminar_pro = get_fresh_query_result("https://internal-redash.federate.frubana.com/",206229,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users_eliminar_pro[['user_id']] = users_eliminar_pro[['user_id']].astype(int)

200


In [4]:
custom = pd.read_csv("push customers - customers.csv")

In [58]:
print(f"HOLA {city}")

HOLA SPO


In [32]:
city = 'SPO'

#users to delete for security checks
users_eliminar = users_eliminar_pro.loc[users_eliminar_pro.region_code == city]['user_id'].unique().tolist()
clientes = custom.loc[(custom.city == city) & (~custom.customer_id.isin(users_eliminar))].customer_id.unique().tolist()

todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
dias = 0 #days to be the offer active

In [33]:
len(clientes)

5218

In [34]:
query = """
SELECT
    s.identifier_value AS city,
    dc.source_id AS customer_id,
    dp.card_id,
    dp.card_description AS product_name,
    COUNT(DISTINCT fs.order_id) AS frequency,
    SUM(fs.product_quantity_x_step_unit) AS demand,
    SUM(fs.gmv_pxq_local)/4.75 AS gmv_usd,
    SUM(COALESCE(fsd.product_discount,0))/4.75 AS discount_applied,
    100.00*discount_applied/gmv_usd AS per_dct,
    (1-(SUM(cogs_p_mtd*fs.product_quantity_x_step_unit)/SUM(fs.product_price*fs.product_quantity_x_step_unit)))*100.00 AS margin,
    gmv_usd*margin/100.00 AS cash_margin,
    cash_margin-discount_applied AS net_cash_margin,
    100.00*net_cash_margin/gmv_usd AS net_margin
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
LEFT JOIN dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date = fs.dim_submitted_date

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND cogs_p_mtd > 0
    AND cat.description NOT IN ('Leite','Arroz')
    AND s.identifier_value = '{city}'
    AND dc.source_id IN {clientes}
    AND DATE(fs.order_submitted_date) > DATE(current_date) - 120

    
GROUP BY 1,2,3,4
""".format(clientes=tuple(set(clientes)), city = city)

data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["frequency", "gmv_usd","demand","net_cash_margin","cash_margin"]] = data_ventas_aux[["frequency", "gmv_usd","demand","net_cash_margin","cash_margin"]].astype('float32')

In [35]:
# Optional to clean the EagleEye
print(len(data_ventas_aux))

61914


In [36]:
df_aux1 = data_ventas_aux.groupby('customer_id').agg({'gmv_usd':np.sum,'cash_margin':np.sum, 'net_cash_margin':np.sum}).reset_index()
df_aux1['margin'] = df_aux1.cash_margin / df_aux1.gmv_usd
df_aux1['net_margin'] = df_aux1.net_cash_margin / df_aux1.gmv_usd

In [37]:
df_aux1.describe()

,customer_id,gmv_usd,cash_margin,net_cash_margin,margin,net_margin
count,5.114000e+03,5114.000000,5114.000000,5114.000000,5114.000000,5114.000000
mean,2.610126e+08,282.608612,45.705032,37.772324,0.173587,0.147074
std,1.240380e+08,516.817688,85.658051,75.421051,0.071899,0.080452
min,1.936076e+07,2.722000,-2.579720,-228.092331,-0.061000,-0.305128
25%,1.431290e+08,60.521700,9.782840,7.572865,0.123691,0.092042
50%,3.454860e+08,130.832001,21.101618,16.665596,0.160568,0.134280
75%,3.667850e+08,294.405128,45.422925,36.783345,0.214900,0.195291
max,3.726699e+08,11634.284180,1771.037964,1685.203735,0.770474,0.758483


In [38]:
data_ventas_aux = data_ventas_aux.loc[data_ventas_aux.customer_id.isin(df_aux1.loc[df_aux1.net_margin > 0.12].customer_id.unique())].reset_index(drop=True)

In [39]:
# Assuming your DataFrame is named 'df'
grouped_df = data_ventas_aux.groupby('customer_id').apply(lambda x: x.nlargest(7, ['frequency', 'gmv_usd'])).reset_index(drop=True)

result_df = grouped_df.groupby('customer_id').agg({'card_id': list, 'product_name': list, 'frequency': list, 'gmv_usd': np.sum, 'cash_margin': np.sum}).reset_index()

result_df["margin"] = 100.00 * result_df.cash_margin / result_df.gmv_usd
result_df['margin'] = np.where(0.65*result_df.margin > 8.5, 8.5,0.65*result_df.margin)
result_df['margin'] = result_df['margin'].apply(lambda x: round(x, 2))
result_df['offer_name'] = ["SH_GRADS_"+city+"_"+todays_date+"_"+str(customer) for customer in result_df.customer_id]

result_df = result_df.loc[result_df.margin > 3].copy()

In [40]:
# Step 1: Convert 'card_id' column from lists to individual values
df_exploded = result_df.explode('card_id')

# Step 2: Count the occurrences of each card ID
card_id_counts = df_exploded['card_id'].value_counts().reset_index()

# Rename columns and set 'card_id' as the index
card_id_counts.rename(columns={'index': 'card_id', 'card_id': 'frequency'}, inplace=True)

# Calculate cumulative percentage and percentage over the total
card_id_counts['cumulative_percentage'] = card_id_counts['frequency'].cumsum() / card_id_counts['frequency'].sum()
card_id_counts['percentage_over_total'] = card_id_counts['frequency'] / card_id_counts['frequency'].sum()

# Multiply by 100 and round to two decimal places
card_id_counts['cumulative_percentage'] = (card_id_counts['cumulative_percentage'] * 100).round(2)
card_id_counts['percentage_over_total'] = (card_id_counts['percentage_over_total'] * 100).round(2)

In [41]:
def concatenate_skus(card_id_list):
    result = []
    for sku in card_id_list:
        try:
            sku_value = frida_products.loc[frida_products['id_tarjeta'] == sku, 'skus'].values[0]
            result.append(sku_value)
        except IndexError:
            result.append('')
    return '#'.join(result)

In [42]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    
    #customers = offer.customer_id.unique()
    customers = offer.customer_id.unique()
    print("Total length: ", len(customers))
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean()/offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [43]:
control,test = create_groups(result_df)

Total length:  2951


In [44]:
final_df = pd.concat([
    pd.DataFrame({'group': 'control', 'customer_id': control}),
    pd.DataFrame({'group': 'test', 'customer_id': test})
], ignore_index=True)

final_df['city'] = city
final_df['todays_date'] = todays_date

In [45]:
final_df.group.value_counts()

test       1476
control    1475
Name: group, dtype: int64

In [46]:
final_df.to_csv("SH_GRADS"+todays_date+city+"GROUPS_9a16.csv", index=False)

In [47]:
global_segment_list = []

for i in result_df.values:
    global_segment_list.append({'name':i[-1], 'customersIds': [i[0]],"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%Y-%m-%d')})
    
result_df = result_df.reset_index(drop=True).copy()

In [48]:
csv_file = {} 
csv_file['offer_name'] = result_df.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = result_df.margin
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 120
csv_file['max_uses_per_customer'] = 2
csv_file['customer_segment_id'] = [1]*len(result_df.offer_name) #[None]
csv_file['sku_id'] = result_df['card_id'].apply(concatenate_skus)
csv_file['segment_behaviour'] = 'INCLUDED'
csv_file['offer_type'] = 'SUPER_HOOK'
csv_file['supplier_discount'] = 0

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [49]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id,segment_behaviour,offer_type,supplier_discount
0,SH_GRADS_SPO_2024-06-20_19766719,None,8.5,true,20-06-2024,22-06-2024,120,2,1,571536,INCLUDED,SUPER_HOOK,0
1,SH_GRADS_SPO_2024-06-20_19767992,None,8.5,true,20-06-2024,22-06-2024,120,2,1,163207,INCLUDED,SUPER_HOOK,0
2,SH_GRADS_SPO_2024-06-20_19771808,None,8.5,true,20-06-2024,22-06-2024,120,2,1,307990#582051#582043#643315#314589#512674#582089,INCLUDED,SUPER_HOOK,0
3,SH_GRADS_SPO_2024-06-20_20179975,None,8.5,true,20-06-2024,22-06-2024,120,2,1,166076#73947#608490#608539#-314737#581787#6084...,INCLUDED,SUPER_HOOK,0
4,SH_GRADS_SPO_2024-06-20_20222914,None,8.5,true,20-06-2024,22-06-2024,120,2,1,235489#412389#108302#73456#611352#240505#124405,INCLUDED,SUPER_HOOK,0


In [50]:
# Replace two or three '#' with one '#'
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('###', '#')
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('##', '#')
    
# Replace '#' at the beginning with an empty string
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('^#', '', regex=True)

# Remove rows where the string is empty
global_offer_csv = global_offer_csv.loc[global_offer_csv['sku_id'] != ""]

In [51]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [52]:
response.text

'Customer segments were created: [774303-SH_GRADS_SPO_2024-06-20_19766719, 774306-SH_GRADS_SPO_2024-06-20_19767992, 774307-SH_GRADS_SPO_2024-06-20_19771808, 774309-SH_GRADS_SPO_2024-06-20_20179975, 774305-SH_GRADS_SPO_2024-06-20_20222914, 774304-SH_GRADS_SPO_2024-06-20_20398872, 774308-SH_GRADS_SPO_2024-06-20_20463224, 774310-SH_GRADS_SPO_2024-06-20_20579095, 774311-SH_GRADS_SPO_2024-06-20_20829977, 774312-SH_GRADS_SPO_2024-06-20_20830204, 774313-SH_GRADS_SPO_2024-06-20_20848817, 774314-SH_GRADS_SPO_2024-06-20_21102052, 774315-SH_GRADS_SPO_2024-06-20_21134817, 774316-SH_GRADS_SPO_2024-06-20_21488857, 774317-SH_GRADS_SPO_2024-06-20_21579097, 774318-SH_GRADS_SPO_2024-06-20_22018553, 774319-SH_GRADS_SPO_2024-06-20_22296651, 774320-SH_GRADS_SPO_2024-06-20_22380430, 774321-SH_GRADS_SPO_2024-06-20_22892148, 774322-SH_GRADS_SPO_2024-06-20_23438840, 774323-SH_GRADS_SPO_2024-06-20_24157016, 774324-SH_GRADS_SPO_2024-06-20_24161918, 774325-SH_GRADS_SPO_2024-06-20_24183117, 774326-SH_GRADS_SPO_202

In [53]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [54]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [55]:
global_offer_csv.discount.describe()

count    2949.000000
mean        8.483184
std         0.135232
min         5.140000
25%         8.500000
50%         8.500000
75%         8.500000
max         8.500000
Name: discount, dtype: float64

In [56]:
global_offer_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2949 entries, 0 to 2950
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   offer_name                    2949 non-null   object 
 1   offer_description             0 non-null      object 
 2   discount                      2949 non-null   float64
 3   automatically_consider_offer  2949 non-null   object 
 4   start_date                    2949 non-null   object 
 5   end_date                      2949 non-null   object 
 6   max_uses_per_order            2949 non-null   int64  
 7   max_uses_per_customer         2949 non-null   int64  
 8   customer_segment_id           2949 non-null   int64  
 9   sku_id                        2949 non-null   object 
 10  segment_behaviour             2949 non-null   object 
 11  offer_type                    2949 non-null   object 
 12  supplier_discount             2949 non-null   int64  
dtypes: 

In [57]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'{todays_date}_SH_FARMING_{city}_OFFER_csv.csv', index = False)